# Проект определения токсичности комментария

Проект оценки токсичности текста

Описание проекта
Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Задача - обучить модель классифицировать комментарии на позитивные и негативные. 
Имеется набор данных с разметкой о токсичности правок.
Необходимо построить модель с целевым значением метрики качества F1 не ниже 0.75. 

Данные находятся в файле toxic_comments.csv.
Столбец text в нём содержит текст комментария, а toxic — целевой признак.

In [1]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.0/22.0 MB 5.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 938.9 kB/s eta 0:00:00a 0:00:01


In [2]:
!pip install spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 3.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.9/491.9 kB 7.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.4/857.4 kB 9.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.6/181.6 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 9.9 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 10.9 MB/s eta 0:00:0000:0100:01


In [11]:
conda install -c conda-forge spacy

Retrieving notices: ...working... done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.1.0
  latest version: 23.3.1

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.3.1



## Package Plan ##

  environment location: /Users/mi_xzm/anaconda3

  added / updated specs:
    - spacy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    catalogue-2.0.8            |  py310h2ec42d9_1          32 KB  conda-forge
    commonmark-0.9.1           |             py_0          46 KB  conda-forge
    confection-0.0.4           |  py310ha188af9_1          62 KB  conda-forge
    cymem-2.0.7                |  py310h7a76584_1          34 KB  conda-forge
    cython-blis-0.7.9          |  py310h936d966_1         5.6 MB  conda-forge
  

In [39]:
from spacy.cli import download
download("en_core_web_trf")
download("en_core_web_sm")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.3/460.3 MB 6.7 MB/s eta 0:00:0000:0100:02
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 8.1 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package v

In [21]:
!pip install 'spacy[transformers]'

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [22]:
import spacy_transformers

## Подготовка

In [69]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.tree import DecisionTreeClassifier
import lightgbm as lgb
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate

import spacy

**Посмотрим на исходные данные.**

In [4]:
data = pd.read_csv('/Users/mi_xzm/Documents/YaPrac/Project_ML_texts/toxic_comments.csv')

<div class="alert alert-warning">
<b>Комментарий ревьюера ⚠️:</b>Да, хорошо, только лучше, чтобы и на платформе ЯП запускался код
    
    
```python
path_yandex = 'datasets/toxic_comments.csv'    
path = 'toxic_comments.csv'

if os.path.exists(path):
    data = pd.read_csv(path, index_col=[0])
    display(data.head(5))
elif os.path.exists(path_yandex):
    data = pd.read_csv(path_yandex, index_col=[0])
    display(data.head(5))
else:
    print('Ошибка в считывании данных')   
```

</div>

<div class="alert alert-info"> <b>Комментарий студента:</b> Согласен, спасибо за замечание, первый раз просто делаю работы не в Юпитере Яндекса :).</div>

In [5]:
data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


**Удалим лишний столбец.**

In [6]:
data = data.drop('Unnamed: 0', axis=1)
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


<div class="alert alert-success">
<b>Комментарий ревьюера ✔️:</b> Хорошо</div>

In [7]:
data['toxic'].mean()

0.10161213369158527

**Токсических комментариев 10% от общего кол-ва в выборке.**

**Посмотрим на соотношение в абсолютных значениях.**

In [8]:
data['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

**Возьмем случайную выборку от исходного датасета, чтобы сохранить в ней аналогичное соотношение классов, как в исходной выборке и сделаем всю работу на ней, т.к. мощностей не хватает для работы на полном датасете, однако, если бы мощностей было достаточно, то работа производилась бы идентичная.**

In [50]:
df = data.sample(50000).reset_index(drop=True)
df['toxic'].mean()

0.1005

**Соотношение классов в новой выборке осталось аналогичным исходной.**

**Проведем подготовку данных, а именно сделаем токенизация через инструменты библиотеки Spacy, параллельно при этом очистим исходный текст от стоп-слов и неалфавитных символов.**

In [51]:
%%time
nlp = spacy.load('en_core_web_sm')#, disable=['parser', 'ner'])

# токенизация, очистка от стоп-слов и символов вне англ. алфавита исходного текста
df['lemm_text'] = df['text'].apply(
    lambda text: 
        " ".join(
            token.lemma_ for token in nlp(text)
                if token.lemma_.lower() not in nlp.Defaults.stop_words and token.is_alpha
        )
)
df

CPU times: user 10min 9s, sys: 3.46 s, total: 10min 12s
Wall time: 10min 14s


,text,toxic,lemm_text
0,"If we don't have valid citations, we shouldn't...",0,valid citation add idea add paragraph clear ne...
1,"dr karl loves himself, fuck yourselves wikiped...",1,dr karl love fuck wikipedia fuck wikipedia
2,I want my page staying clean - if you dont lik...,0,want page stay clean like ban
3,I may not be a member of a particular wiki pro...,0,member particular wiki project work profession...
4,"Sorry, but I still believe you are mistaken. ...",0,sorry believe mistaken force people reveal edi...
...,...,...,...
49995,U R A FUCKIN MUSLIM ARENT U?,1,u r FUCKIN MUSLIM arent u
49996,Vergina Sun \n\nΤετρακτύς εν δρασει!\n\nThe Ve...,0,Vergina Sun Τετρακτύς εν δρασει Vergina Sun an...
49997,I have finally got around to reading the talk ...,0,finally read talk page believe inclusion Forbe...
49998,"If it isn't Gunga-Din, the loyal water bearer ...",0,Gunga Din loyal water bearer janitor offer ser...


<div class="alert alert-success">
<b>Комментарий ревьюера ✔️:</b>
Так же можешь попробовать сделай вот так используя spacy:
    
```python
import spacy
nlp = spacy.load('en_core_web_sm')

def lemmatize_spacy(text):
    return " ".join([token.lemma_ for token in nlp(text)])

data['lemm_spacy'] = data['text'].progress_apply(lemmatize_spacy)     
```

Вот так можно следить за выполнением функции (как раз выше его использовал)

```python
    
from tqdm import tqdm
tqdm.pandas()
data['text'] = data['text'].progress_apply(lemmatize_text) 
```
Где *lemmatize_text* - функция

    
P.S Кстати можно ускорить spacy ( раз в 5 +-)
    
```python
new_corpus = []

for doc in tqdm(nlp.pipe(data['text'], batch_size=64, n_process=-1, disable=["parser", "ner"]), total=len(data['text'])):
    word_list = [tok.lemma_ for tok in doc]
    new_corpus.append(' '.join(word_list))
    
data['lemm_spacy_new'] = new_corpus   
    
    
```
    
Для больших объемов текста ```SpaCy``` рекомендует использовать ```nlp.pipe```, который может работать в пакетах ```batch_size```(Допустим ```batch_size=64```, тогда все наши данные делятся на 64 пакета (как прям в фолдах, только там мы пишем число на сколько поделить наши данные (3/5/10 частей),а тут пишем, сколько данных отдать в обработку)  и имеет встроенную поддержку многопроцессорной обработки ```n_process``` (аналог ```n_jobs``` для GridSearchCV).

Кроме того, нужно убедиться, что мы отключили все элементы конвейера, которые мы не планируем использовать, поскольку они просто потратят время на обработку. Если мы выполняем только лемматизацию, то необходимо передать ```disable=["parser", "ner"]``` к ```nlp.pipe```.
    
    
P.S.S Если запустишь ```nlp.pipe```, то первое время нужно подождать прежде, чем начнется обработка 
</div>

<div class="alert alert-info"> <b>Комментарий студента:</b> Не совсем понял по первому пункту, где progress_apply: это функция этого самого tqdm для прогресс-бара процесса исполнения кода данной строки? Если да, то что означает вот это: tqdm.pandas()? Т.е. хочется понять принцип применения этого tqdm, мы должны оборачивать этой функцией какую часть кода и как именно, видел в цикле for ты обернул ей список, если я правильно понял.

Про nlp.pipe дополнительно почитаю, спасибо.</div>

<div class="alert alert-success">
<b>Комментарий ревьюера V2✔️:</b>

tqdm.pandas() мы так объявляем, что будем использовать tqdm для пандас. Метод progress_apply очень похож на метод apply, но добавляет индикатор прогресса, который показывает, сколько времени осталось до завершения операции и сколько времени уже прошло.
    
+  https://tqdm.github.io/docs/tqdm/
    
    
Еще можно tqdm использовать для циклов показывая прогресс бар 

+  https://tqdm.github.io/
    
P.S progress_apply это просто уже tqdm сам допил, чтобы было удобнее наблюдать;)
</div>

**Разобъем нашу выборку на тренировочную и тестовую. Валидационной не будет, т.к. будем использовать кросс-валидацию.**

In [52]:
#разбиваем признаки и целевые признаки на тренировочную и тестовую выборки в соотношении 80/20
train, test = train_test_split(df, test_size=0.2, random_state=1)
train.shape

(40000, 3)

In [53]:
test.shape

(10000, 3)

**Создадим теперь корпус для тестовой и тренировочной выборок, применим TF_IDF векторизатор для получения фичей из исходного текста для тренировочной выборки, а затем на их основе создадим фичи и для тестовой выборки.**

In [54]:
%%time
corpus_train = train['lemm_text']
corpus_test = test['lemm_text']

count_tf_idf_train = TfidfVectorizer()

features_train = count_tf_idf_train.fit_transform(corpus_train)
target_train = train['toxic']
features_test = count_tf_idf_train.transform(corpus_test)
target_test = test['toxic']

CPU times: user 881 ms, sys: 10.6 ms, total: 892 ms
Wall time: 894 ms


<div class="alert alert-success">
<b>Комментарий ревьюера ✔️:</b> Отлично, молодец, верно используешь Tfidf.
    
    
Совет: Внутри кросс-валидации происходит разбиение выборки на train и valid. Однако, в таком случае векторизатор обучен на всей выборке(train), а это не совсем корректно. Чтобы избежать это можно воспользоваться Pipeline:
    
```python
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stopwords)),
    ('logreg', LogisticRegression(random_state=42)),
])
parameters = {
    'tfidf__max_df': (0.25, 0.5, 0.75),
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'logreg__C': [1,2,6]
}

grid_search_tune = RandomizedSearchCV(pipeline, parameters, cv=3, n_jobs=-1, scoring='f1', verbose=3)
grid_search_tune.fit(train_features, train_targets)
  
    
```
    
Это просто каркас, можешь сам выбрать какие параметры использовать для подбора:) 
    
+  https://runebook.dev/ru/docs/scikit_learn/modules/generated/sklearn.model_selection.halvinggridsearchcv - тут про HalvingGridSearchCV
    
+  https://www.rupython.com/python-sklearn-pipeline-pipeline-28301.html - про pipeline
+  https://towardsdatascience.com/how-to-use-sklearn-pipelines-for-ridiculously-neat-code-a61ab66ca90d

</div>

<div class="alert alert-info"> <b>Комментарий студента:</b> Спасибо, про пайплайны почитаю, слышал о них, но ранее не использовал. По поводу обучения на всем трейне, в который включается валидационная часть - абсолютно согласен. Имеет ли смысл использовать тогда просто разбиение на отдельно трейн и валид выборки, если не хотим использовать пайплайн для получения более "честного" результата? </div>

<div class="alert alert-success">
<b>Комментарий ревьюера V2✔️:</b> Имеешь ввиду, что если хочешь отказаться от pipeline, но при этом делать на три выборки, чтобы после кросс валидации еще и провалидироваться? Это не имеет смысла, так как валидационная выборка будет выступать в роли теста по сути (мы два раза проведем тест) + жалко данные(которые почти никак не используются) + кросс валидации гораздо более показательна, так как мы берем среднее
    
    
+  https://academy.yandex.ru/handbook/ml/article/kross-validaciya
+  https://academy.yandex.ru/handbook/ml/article/podbor-giperparametrov
</div>

## Обучение

### Модель классификатор решающего дерева

**Будем использовать GridSearchCV со встроенной кросс-валидацией для поиска наилучшего результата метрики f1, которую мы используем в качестве оценки нашей модели по условию задачи.**

In [67]:
%%time
model_tree = DecisionTreeClassifier(random_state=1)
param = {
         'criterion': ['gini', 'entropy'],
         'max_depth': range(1, 10, 3)
        }
gridsearch_tree = GridSearchCV(
    estimator=model_tree, 
    param_grid=param, 
    scoring='f1', 
    cv=3)
gridsearch_tree.fit(features_train, target_train)
gridsearch_tree.best_params_

CPU times: user 17.5 s, sys: 163 ms, total: 17.6 s
Wall time: 17.6 s


{'criterion': 'gini', 'max_depth': 7}

**Посмотрим теперь, какой наилучший результат метрики f1 удалось достигнуть данной моделью.**

In [68]:
f1_train_tree = round(gridsearch_tree.best_score_, 3)
f1_train_tree

0.547

### Модель классификатор CatBoost

**Будем использовать GridSearchCV со встроенной кросс-валидацией для поиска наилучшего результата метрики f1, которую мы используем в качестве оценки нашей модели по условию задачи. Выведем это значение метрики f1.**

In [61]:
%%time
model = CatBoostClassifier(silent=True, random_state=1)
param = {}
gridsearch = GridSearchCV(
    estimator=model, 
    param_grid=param, 
    scoring='f1', 
    cv=3)
gridsearch.fit(features_train, target_train)

CPU times: user 30min 4s, sys: 35.1 s, total: 30min 39s
Wall time: 5min 56s


0.691

<div class="alert alert-success">
<b>Комментарий ревьюера ✔️:</b> Для Catboost будет лучше на самом деле воспользовать внутренним кодированием:
    
    
+  https://colab.research.google.com/drive/1bQOXIKpeyGZgda2qQqDijE2SIbSGX7-Z?usp=sharing    
</div>

<div class="alert alert-info"> <b>Комментарий студента:</b> Имеешь в виду грузить сразу "сырой" текст в него? Или всё-таки необходимо его сначала очистить от спец-символов и стоп-слов?

Насколько кстати имеет смысл делать gridsearchCV для CatBoost'a с точки зрения подбора гиперпараметров? Слышал, что данная модель при жонглировании гиперпараметрами позволяет лишь чуть улучшить метрику, что справедливо для всяких кагловских соревнований только. Так ли это по твоему опыту?</div>

<div class="alert alert-success">
<b>Комментарий ревьюера V2✔️:</b> Можно поэксперементировать, загрузить сырой или обработать 
    
    
Да, в приницпе CatBoost работает из коробки уже хорошо, единственное стоит подобрать learning rate и кол-во итераций (чтобы модель точно сошлась)
</div>

<div class="alert alert-success">
<b>Комментарий ревьюера ✔️:</b> Можешь воспользоваться кросс валидацией cross_val_score</div>

## Анализ моделей

**Лучше из данных двух моделей проявила себя модель CatBoost. Проверим теперь, какой результат метрики f1 данная модель покажет на тестовой выборке.**

In [62]:
%%time
predictions_test = gridsearch.best_estimator_.predict(features_test)
f1_test = f1_score(target_test, predictions_test)
f1_test

CPU times: user 188 ms, sys: 8.69 ms, total: 197 ms
Wall time: 77 ms


0.7565011820330969

**Удалось получить на тестовой выборке значение метрики f1 выше 0.75, что удовлетворяет исходной задаче.**

## Проверка моделей на адекватность

**Проверим модель на адекватность с помощью Dummy Model из класса sklearn, предварительно импортировав ее в первом пункте.**

**Проверка состоит в том, чтобы качество выбранной выше обученной модели было выше, чем качество модели Dummy, которая предсказывает результат не опираясь на признаки тренировочной выборки.**

In [73]:
model_dummy = DummyClassifier(random_state=10)
parameters_dummy = {'strategy':['most_frequent', 'prior', 'stratified', 'uniform'],
                   }
gridsearch_dummy = GridSearchCV(
    estimator=model_dummy, 
    param_grid=parameters_dummy, 
    scoring='f1', 
    cv=3, n_jobs=-1)
gridsearch_dummy.fit(features_train, target_train)
print('Лучшее значение метрики f1 для Dummy-модели на тренировочной выборке =', gridsearch_dummy.best_score_)

Лучшее значение метрики f1 для Dummy-модели на тренировочной выборке = 0.16992010542788896


In [76]:
predict_dummy = gridsearch_dummy.best_estimator_.predict(features_test)
print('Лучшее значение метрики f1 для Dummy-модели на тестовой выборке =', f1_score(target_test, predict_dummy))

Лучшее значение метрики f1 для Dummy-модели на тестовой выборке = 0.15450785773366418


## Выводы

**В итоге с помощью Dummy модели удалось получить наилучшее качество модели с f1 равным 0.16 на тестовой выборке, что значительно ниже наилучшего результата выбранной и обученной модели CatBoost. Что доказывает адекватность найденной и выбранной нами модели.**

<div class="alert alert-success">
<b>Комментарий ревьюера ✔️:</b> 

Для работы с текстами используют и другие подходы. Например, сейчас активно используются RNN (LSTM) и трансформеры (BERT и другие с улицы Сезам, например, ELMO). НО! Они не являются панацеей, не всегда они нужны, так как и TF-IDF или Word2Vec + модели из классического ML тоже могут справляться. \
BERT тяжелый, существует много его вариаций для разных задач, есть готовые модели, есть надстройки над библиотекой transformers. Если, обучать BERT на GPU (можно в Google Colab или Kaggle), то должно быть побыстрее.\
https://huggingface.co/transformers/model_doc/bert.html \
https://t.me/renat_alimbekov \
https://colah.github.io/posts/2015-08-Understanding-LSTMs/ - Про LSTM \
https://web.stanford.edu/~jurafsky/slp3/10.pdf - про энкодер-декодер модели, этеншены\
https://pytorch.org/tutorials/beginner/transformer_tutorial.html - официальный гайд
по трансформеру от создателей pytorch\
https://transformer.huggingface.co/ - поболтать с трансформером \
Библиотеки: allennlp, fairseq, transformers, tensorflow-text — множество реализованных
методов для трансформеров методов NLP \
Word2Vec https://radimrehurek.com/gensim/models/word2vec.html 
    
</div>